# Make BIDS configuration files
Natalia Vélez, August 2021

In [1]:
import urllib, json, yaml, sys
import pandas as pd 
import numpy as np

# Helper functions
sys.path.append('..')
from utils import str_extract, int_extract 

Sessions:

In [2]:
with open('session_labels.txt', 'r') as f:
    sessions = f.read().splitlines()
    
print(*sessions, sep='\n')

210812_teaching_01
210812_teaching_02
210816_teaching_03
210816_teaching_04
210821_teaching_05
210821_teaching_06
210822_teaching_07
210822_teaching_08
210823_teaching_09
210823_teaching_10
210823_teaching_11
210823_teaching_12
210824_teaching_13
210824_teaching_14
210825_teaching_15
210825_teaching_16
210825_teaching_17
210826_teaching_18
210826_teaching_19
210826_teaching_20
210827_teaching_21
210827_teaching_22
210829_teaching_23
210829_teaching_24
210829_teaching_25
210830_teaching_26
210830_teaching_27
210831_teaching_28
210831_teaching_29
210831_teaching_30


Read credentials:

In [3]:
with open('auth.key', 'r') as f:
    (user, pw) = f.read().splitlines()

FileNotFoundError: [Errno 2] No such file or directory: 'auth.key'

Read data from XNAT

In [ ]:
def load_session(s):
    url_template = 'https://cbscentral.rc.fas.harvard.edu/data/projects/Gershman/subjects/%s/experiments/%s/scans?format=json'
    url = url_template % (s,s)
    p = urllib.request.HTTPPasswordMgrWithDefaultRealm()
    p.add_password(None, url, user, pw)

    handler = urllib.request.HTTPBasicAuthHandler(p)
    opener = urllib.request.build_opener(handler)
    urllib.request.install_opener(opener)

    response = urllib.request.urlopen(url).read()
    page = json.loads(response)
    session_data = pd.DataFrame(page['ResultSet']['Result'])
    
    return session_data

Helper function: Fix fmap inputs

In [ ]:
def intended_for(s, func):
    func_runs = s.split(',')
    func_runs = [int(r) for r in func_runs]
    
    func_subset = func[func.scan.isin(func_runs)]    
    return func_subset.id.values.tolist()

Main function: Turn XNAT session info into YAML config file:

In [ ]:
def session_config(s):
    # load inputs
    session_data = load_session(s)
    session_data = session_data[['ID', 'series_description', 'note']]
    session_data['ID'] = session_data.ID.astype(int)

    # prepare outputs
    out_file = 'outputs/config/%s.yaml' % s
    config_data = {'anat': [], 'func': {'bold': []}} # initialize

    # anatomical
    anat_scans = session_data[session_data.note.str.contains('anat')]
    config_data['anat'] = {'T1w': {'scan': int(anat_scans['ID'].values[0]), 'run': 1}}

    # functional files
    func_scans = session_data[session_data.note.str.contains('teaching|tomloc')]
    func_scans['run'] = func_scans.groupby('note').cumcount()+1
    func_scans['id'] = func_scans.apply(lambda row: 'task-%s_run-%02d' % (row['note'], row['run']), axis=1)
    func_scans = func_scans.rename(columns={'ID': 'scan', 'note': 'task'})
    func_scans = func_scans[['scan', 'task', 'run', 'id']]
    func_dict = func_scans.to_dict(orient='records')
    config_data['func']['bold'] = func_dict

    # fieldmaps
    fmap_scans = session_data[session_data.series_description.str.contains('FieldMap')]
    if fmap_scans.shape[0] > 1:
        config_data['fmap'] = {}

        fmap_scans = fmap_scans.reset_index(drop=True)
        fmap_scans['run'] = fmap_scans.index+1
        fmap_scans['direction'] = fmap_scans.series_description.str.extract('(AP|PA)')
        fmap_scans['note'] = fmap_scans.note.apply(lambda s: intended_for(s, func_scans))
        fmap_scans = fmap_scans.rename(columns={'ID': 'scan', 'note': 'intended_for'})
        fmap_scans = fmap_scans[['scan', 'run', 'direction', 'intended_for']]
        fmap_scans = fmap_scans.to_dict(orient='records')
        config_data['fmap']['epi'] = fmap_scans

    # save to file
    with open(out_file, 'w') as out:
        yaml.dump(config_data, out)
        
    return config_data

Loop through sessions and save outputs:

In [ ]:
all_sessions = {s:session_config(s) for s in sessions}

Quality checks:

In [ ]:
qa_list = []

for sub,ses in all_sessions.items():

    # unpacking session information
    func_ids = [run['id'] for run in ses['func']['bold']]

    # quality checks
    n_teaching_runs = len([s for s in ses['func']['bold'] if s['task'] == 'teaching'])
    n_tomloc_runs = len([s for s in ses['func']['bold'] if s['task'] == 'tomloc'])
    has_anat = 'anat' in ses
    has_fmap = 'fmap' in ses
    if has_fmap:
        n_fmaps = int(len(ses['fmap']['epi'])/2)
        fmap_func_ids = np.unique([run for fmap in ses['fmap']['epi'] for run in fmap['intended_for']]).tolist()
        fmaps_assigned = func_ids == fmap_func_ids
    else:
        n_fmaps = 0
        fmaps_assigned = False

    qa_list.append((sub, n_teaching_runs, n_tomloc_runs, has_anat, has_fmap, n_fmaps, fmaps_assigned))
    
qa_df = pd.DataFrame(qa_list,
                    columns = ('sub', 'n_teaching_runs', 'n_tomloc_runs', 'has_anat',
                               'has_fmap', 'n_fmaps', 'fmaps_assigned'))

qa_df.to_csv('outputs/protocol_qa.csv', index=False)